In [ ]:
import sagemaker
import boto3

from sagemaker.pytorch.model import PyTorchModel
from sagemaker.huggingface.model import HuggingFaceModel, HuggingFacePredictor
from sagemaker.serverless import ServerlessInferenceConfig
from sagemaker.serializers import DataSerializer

import io
from PIL import Image
import torch
import numpy as np

In [ ]:
boto3_sess = boto3.Session(profile_name=os.environ.get["SAGEMAKER_PROFILE"])
sess = sagemaker.Session(boto_session=boto3_sess)

role = os.environ.get["SAGEMAKER_ROLE"]

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")

In [ ]:
repository = "hustvl/yolos-tiny"
model_id=repository.split("/")[-1]
s3_location=f"s3://{sess.default_bucket()}/custom_inference/{model_id}/model.tar.gz"


In [ ]:
!git lfs install
!git clone https://huggingface.co/$repository

In [ ]:
!cp -r code/ $model_id/code/

In [ ]:
%cd $model_id
!tar zcvf model.tar.gz *

In [ ]:
!aws s3 --profile celapp cp model.tar.gz $s3_location

In [ ]:
image_serializer = DataSerializer(content_type='image/x-image') 

huggingface_model = HuggingFaceModel(
    model_data=s3_location, # path to your model and script
    sagemaker_session=sess,
    role=role,                    # iam role with permissions to create an Endpoint
    transformers_version="4.17",  # transformers version used
    pytorch_version="1.10",        # pytorch version used
    py_version='py38',            # python version used
)

# Specify MemorySizeInMB and MaxConcurrency in the serverless config object
serverless_config = ServerlessInferenceConfig(
    memory_size_in_mb=4096, max_concurrency=10,
)

# deploy the endpoint endpoint
yolos_predictor = huggingface_model.deploy(
    endpoint_name="yolos-t-object-detection-serverless",
    serverless_inference_config=serverless_config,
    serializer=image_serializer
)

In [ ]:
%cd ..

In [ ]:
data = {
  "inputs": "example_resized.jpg",
}

res = yolos_predictor.predict(data="example_resized.jpg")

In [ ]:
yolos_predictor.delete_model()
yolos_predictor.delete_endpoint()

In [ ]:
import matplotlib.pyplot as plt
from transformers import YolosForObjectDetection

# colors for visualization
COLORS = [[0.000, 0.447, 0.741], [0.850, 0.325, 0.098], [0.929, 0.694, 0.125],
          [0.494, 0.184, 0.556], [0.466, 0.674, 0.188], [0.301, 0.745, 0.933]]
model = YolosForObjectDetection.from_pretrained("hustvl/yolos-small")

def plot_results(pil_img, prob, boxes):
    plt.figure(figsize=(16,10))
    plt.imshow(pil_img)
    ax = plt.gca()
    colors = COLORS * 100
    for p, (xmin, ymin, xmax, ymax), c in zip(prob, boxes, colors):
        ax.add_patch(plt.Rectangle((xmin, ymin), xmax - xmin, ymax - ymin,
                                   fill=False, color=c, linewidth=3))
        cl = p.argmax()
        text = f'{model.config.id2label[cl.item()]}: {p[cl]:0.2f}'
        ax.text(xmin, ymin, text, fontsize=15,
                bbox=dict(facecolor='yellow', alpha=0.5))
    plt.axis('off')
    plt.show()
    

In [ ]:
image = Image.open("example_resized.jpg")
plot_results(image, np.asarray(res["probabilities"]), res["bounding_boxes"])